# Chapter 5 Data Preprocessing and Cleaning

In [1]:
import glob
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.formula.api import ols

Configuring directory parameters

In [7]:
input_files = glob.glob('Data/cars_*.csv')
output_file = 'Data/data_combined_quarterly_fuel_stats.csv'

Define fuel type mapping

In [8]:
fuel_mapping = {
    'petrol': 'petrol',
    'petrol_ng': 'petrol',
    'greendiesel': 'greendiesel',
    'hybrid_petrol': 'hybrid_petrol',
    'diesel': 'greendiesel',
    'electric': 'electric'
}

Store data for all years

In [9]:
df_vehicle_sales = []

Process each file

In [10]:
for file in input_files:
    # Extract the year from the file name
    year = file.split('_')[1].split('.')[0]
    
    # Read data
    df = pd.read_csv(file)
    
    # Handling date and fuel type
    df['date_reg'] = pd.to_datetime(df['date_reg'])
    df['quarter'] = df['date_reg'].dt.quarter
    df['fuel'] = df['fuel'].map(fuel_mapping).fillna('other')
    
    # Statistics by quarter and fuel type
    quarterly_stats = df.groupby(['quarter', 'fuel']).size().unstack(fill_value=0)
    
    # Keep only the fuel types we care about
    fuel_types = ['petrol', 'greendiesel', 'hybrid_petrol', 'electric']
    quarterly_stats = quarterly_stats.reindex(columns=fuel_types, fill_value=0)
    
    # Add year information
    quarterly_stats['year'] = year
    df_vehicle_sales.append(quarterly_stats)

Combine data from all years

In [11]:
df_combined = pd.concat(df_vehicle_sales)

Reset index and rename quarters

In [12]:
df_combined = df_combined.reset_index()
df_combined['quarter'] = 'Q' + df_combined['quarter'].astype(str)

Rearrange the order of columns

In [13]:
column_order = ['year', 'quarter'] + fuel_types
df_combined = df_combined[column_order]

Save the results as csv file

In [14]:
df_combined.to_csv(output_file, index=False)

Preview the statistical data

In [15]:
df_combined

fuel,year,quarter,petrol,greendiesel,hybrid_petrol,electric
0,2017,Q1,130462,11323,1845,9
1,2017,Q2,132040,11615,1621,13
2,2017,Q3,127712,11177,2175,2
3,2017,Q4,132674,13448,3502,1
4,2018,Q1,124378,11774,2519,4
5,2018,Q2,143169,12124,2594,2
6,2018,Q3,152348,17695,2336,2
7,2018,Q4,128473,11257,294,2
8,2019,Q1,132618,10661,1164,5
9,2019,Q2,143735,10535,1537,0


In [16]:
df_combined.head(8)

fuel,year,quarter,petrol,greendiesel,hybrid_petrol,electric
0,2017,Q1,130462,11323,1845,9
1,2017,Q2,132040,11615,1621,13
2,2017,Q3,127712,11177,2175,2
3,2017,Q4,132674,13448,3502,1
4,2018,Q1,124378,11774,2519,4
5,2018,Q2,143169,12124,2594,2
6,2018,Q3,152348,17695,2336,2
7,2018,Q4,128473,11257,294,2


In [17]:
# Filter rows where the first column 'series_type' is 'level'
df_fuel_price_level = df_fuel_price.copy()
df_fuel_price_level = df_fuel_price_level[df_fuel_price_level['series_type'] == 'level']

# Convert the 'date' column to datetime format
df_fuel_price_level['date'] = pd.to_datetime(df_fuel_price_level['date'])

# Extract year and quarter number into separate columns
df_fuel_price_level['year'] = df_fuel_price_level['date'].dt.year
df_fuel_price_level['quarter'] = 'Q' + df_fuel_price_level['date'].dt.quarter.astype(str)

# Keep only the price columns to calculate averages
price_columns = ['ron95', 'ron97', 'diesel', 'diesel_eastmsia']

# Group by year and quarter, then calculate mean for each fuel type
df_average = df_fuel_price_level.groupby(['year', 'quarter'])[price_columns].mean().round(2).reset_index()

# Save the result to a CSV file
df_average.to_csv('Data/data_quarterly_average_fuel_prices.csv', index=False)

# Preview the result (only 'level' rows)
df_average.head(8)

,year,quarter,ron95,ron97,diesel,diesel_eastmsia
0,2017,Q1,2.13,2.41,2.11,2.11
1,2017,Q2,2.09,2.36,2.02,2.02
2,2017,Q3,2.10,2.37,2.03,2.03
3,2017,Q4,2.26,2.54,2.19,2.19
4,2018,Q1,2.24,2.51,2.24,2.24
5,2018,Q2,2.20,2.52,2.18,2.18
6,2018,Q3,2.20,2.61,2.18,2.18
7,2018,Q4,2.20,2.70,2.18,2.18
